In [1]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

In [2]:
class CFG:
    seed = 42
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'

In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [4]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

In [5]:
train = pd.read_parquet("../tmp/train_agg.parquet")
test = pd.read_parquet("../tmp/test_agg.parquet")
print(train.shape)

(458913, 1075)


In [6]:
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]
cat_features = [f"{cf}_last" for cf in cat_features]
cat_features

['B_30_last',
 'B_38_last',
 'D_114_last',
 'D_116_last',
 'D_117_last',
 'D_120_last',
 'D_126_last',
 'D_63_last',
 'D_64_last',
 'D_66_last',
 'D_68_last']

In [7]:
for cat_col in tqdm(cat_features):
    encoder = LabelEncoder()
    train[cat_col] = encoder.fit_transform(train[cat_col])
    test[cat_col] = encoder.transform(test[cat_col])

  0%|          | 0/11 [00:00<?, ?it/s]

In [8]:
num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
last_num_cols = [col for col in num_cols if ('last' in col) or ('first' in col)]
for col in last_num_cols:
    train[col + '_round2'] = train[col].round(2)
    test[col + '_round2'] = test[col].round(2)

In [9]:
train

,customer_ID,P_2_last,P_2_mean,P_2_std,P_2_min,P_2_max,D_39_last,D_39_mean,D_39_std,D_39_min,D_39_max,B_1_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_2_last,B_2_mean,B_2_std,B_2_min,B_2_max,R_1_last,R_1_mean,R_1_std,R_1_min,R_1_max,S_3_last,S_3_mean,S_3_std,S_3_min,S_3_max,D_41_last,D_41_mean,D_41_std,D_41_min,D_41_max,B_3_last,B_3_mean,B_3_std,B_3_min,B_3_max,D_42_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_43_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_44_last,D_44_mean,D_44_std,D_44_min,D_44_max,B_4_last,B_4_mean,B_4_std,B_4_min,B_4_max,D_45_last,D_45_mean,D_45_std,D_45_min,D_45_max,B_5_last,B_5_mean,B_5_std,B_5_min,B_5_max,R_2_last,R_2_mean,R_2_std,R_2_min,R_2_max,D_46_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_47_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_48_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_49_last,D_49_mean,D_49_std,D_49_min,D_49_max,B_6_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_7_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_8_last,B_8_mean,B_8_std,B_8_min,B_8_max,D_50_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_51_last,D_51_mean,D_51_std,D_51_min,D_51_max,B_9_last,B_9_mean,B_9_std,B_9_min,B_9_max,R_3_last,R_3_mean,R_3_std,R_3_min,R_3_max,D_52_last,D_52_mean,D_52_std,D_52_min,D_52_max,P_3_last,P_3_mean,P_3_std,P_3_min,P_3_max,B_10_last,B_10_mean,B_10_std,B_10_min,B_10_max,D_53_last,D_53_mean,D_53_std,D_53_min,D_53_max,S_5_last,S_5_mean,S_5_std,S_5_min,S_5_max,B_11_last,B_11_mean,B_11_std,B_11_min,B_11_max,S_6_last,S_6_mean,S_6_std,S_6_min,S_6_max,D_54_last,D_54_mean,D_54_std,D_54_min,D_54_max,R_4_last,R_4_mean,R_4_std,R_4_min,R_4_max,S_7_last,S_7_mean,S_7_std,S_7_min,S_7_max,B_12_last,B_12_mean,B_12_std,B_12_min,B_12_max,S_8_last,S_8_mean,S_8_std,S_8_min,S_8_max,D_55_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_56_last,D_56_mean,D_56_std,D_56_min,D_56_max,B_13_last,B_13_mean,B_13_std,B_13_min,B_13_max,R_5_last,R_5_mean,R_5_std,R_5_min,R_5_max,D_58_last,D_58_mean,D_58_std,D_58_min,D_58_max,S_9_last,S_9_mean,S_9_std,S_9_min,S_9_max,B_14_last,B_14_mean,B_14_std,B_14_min,B_14_max,D_59_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_60_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_61_last,D_61_mean,D_61_std,D_61_min,D_61_max,B_15_last,B_15_mean,B_15_std,B_15_min,B_15_max,S_11_last,S_11_mean,S_11_std,S_11_min,...,B_31_diff1,S_19_diff1,R_19_diff1,B_32_diff1,S_20_diff1,R_20_diff1,R_21_diff1,B_33_diff1,D_89_diff1,R_22_diff1,R_23_diff1,D_91_diff1,D_92_diff1,D_93_diff1,D_94_diff1,R_24_diff1,R_25_diff1,D_96_diff1,S_22_diff1,S_23_diff1,S_24_diff1,S_25_diff1,S_26_diff1,D_102_diff1,D_103_diff1,D_104_diff1,D_105_diff1,D_106_diff1,D_107_diff1,B_36_diff1,B_37_diff1,R_26_diff1,R_27_diff1,D_108_diff1,D_109_diff1,D_110_diff1,D_111_diff1,B_39_diff1,D_112_diff1,B_40_diff1,S_27_diff1,D_113_diff1,D_115_diff1,D_118_diff1,D_119_diff1,D_121_diff1,D_122_diff1,D_123_diff1,D_124_diff1,D_125_diff1,D_127_diff1,D_128_diff1,D_129_diff1,B_41_diff1,B_42_diff1,D_130_diff1,D_131_diff1,D_132_diff1,D_133_diff1,R_28_diff1,D_134_diff1,D_135_diff1,D_136_diff1,D_137_diff1,D_138_diff1,D_139_diff1,D_140_diff1,D_141_diff1,D_142_diff1,D_143_diff1,D_144_diff1,D_145_diff1,target,P_2_last_round2,D_39_last_round2,B_1_last_round2,B_2_last_round2,R_1_last_round2,S_3_last_round2,D_41_last_round2,B_3_last_round2,D_42_last_round2,D_43_last_round2,D_44_last_round2,B_4_last_round2,D_45_last_round2,B_5_last_round2,R_2_last_round2,D_46_last_round2,D_47_last_round2,D_48_last_round2,D_49_last_round2,B_6_last_round2,B_7_last_round2,B_8_last_round2,D_50_last_round2,D_51_last_round2,B_9_last_round2,R_3_last_round2,D_52_last_round2,P_3_last_round2,B_10_last_round2,D_53_last_round2,S_5_last_round2,B_11_last_round2,S_6_last_round2,D_54_last_round2,R_4_last_round2,S_7_last_round2,B_12_last_round2,S_8_last_round2,D_55_last_round2,D_56_last_round2,B_13_last_round2,R_5_last_round2,D_58_last_round2,S_9_last_round2,B_14_last_round2,D_59_last_round2,D_60_last_round2,D_61_last_round2,B_15_last_round2,S_11_last_round2,D_62_last_round2,D_65_last_round2,B_16_last_round2,B_17_last_round2,B_18_last_round2,B

In [10]:
num_cols = [col for col in train.columns if 'last' in col]
num_cols = [col[:-5] for col in num_cols if 'round' not in col]
for col in num_cols:
    try:
        train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
        test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
    except Exception as ex:
        print(col, ex)

B_30 'B_30_mean'
B_38 'B_38_mean'
D_114 'D_114_mean'
D_116 'D_116_mean'
D_117 'D_117_mean'
D_120 'D_120_mean'
D_126 'D_126_mean'
D_63 'D_63_mean'
D_64 'D_64_mean'
D_66 'D_66_mean'
D_68 'D_68_mean'


In [11]:
%%time
num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
train[num_cols] = train[num_cols].astype(np.float16)
test[num_cols] = test[num_cols].astype(np.float16)

CPU times: user 6min 22s, sys: 7min 26s, total: 13min 49s
Wall time: 13min 49s


In [12]:
features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]

In [13]:
train

,customer_ID,P_2_last,P_2_mean,P_2_std,P_2_min,P_2_max,D_39_last,D_39_mean,D_39_std,D_39_min,D_39_max,B_1_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_2_last,B_2_mean,B_2_std,B_2_min,B_2_max,R_1_last,R_1_mean,R_1_std,R_1_min,R_1_max,S_3_last,S_3_mean,S_3_std,S_3_min,S_3_max,D_41_last,D_41_mean,D_41_std,D_41_min,D_41_max,B_3_last,B_3_mean,B_3_std,B_3_min,B_3_max,D_42_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_43_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_44_last,D_44_mean,D_44_std,D_44_min,D_44_max,B_4_last,B_4_mean,B_4_std,B_4_min,B_4_max,D_45_last,D_45_mean,D_45_std,D_45_min,D_45_max,B_5_last,B_5_mean,B_5_std,B_5_min,B_5_max,R_2_last,R_2_mean,R_2_std,R_2_min,R_2_max,D_46_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_47_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_48_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_49_last,D_49_mean,D_49_std,D_49_min,D_49_max,B_6_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_7_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_8_last,B_8_mean,B_8_std,B_8_min,B_8_max,D_50_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_51_last,D_51_mean,D_51_std,D_51_min,D_51_max,B_9_last,B_9_mean,B_9_std,B_9_min,B_9_max,R_3_last,R_3_mean,R_3_std,R_3_min,R_3_max,D_52_last,D_52_mean,D_52_std,D_52_min,D_52_max,P_3_last,P_3_mean,P_3_std,P_3_min,P_3_max,B_10_last,B_10_mean,B_10_std,B_10_min,B_10_max,D_53_last,D_53_mean,D_53_std,D_53_min,D_53_max,S_5_last,S_5_mean,S_5_std,S_5_min,S_5_max,B_11_last,B_11_mean,B_11_std,B_11_min,B_11_max,S_6_last,S_6_mean,S_6_std,S_6_min,S_6_max,D_54_last,D_54_mean,D_54_std,D_54_min,D_54_max,R_4_last,R_4_mean,R_4_std,R_4_min,R_4_max,S_7_last,S_7_mean,S_7_std,S_7_min,S_7_max,B_12_last,B_12_mean,B_12_std,B_12_min,B_12_max,S_8_last,S_8_mean,S_8_std,S_8_min,S_8_max,D_55_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_56_last,D_56_mean,D_56_std,D_56_min,D_56_max,B_13_last,B_13_mean,B_13_std,B_13_min,B_13_max,R_5_last,R_5_mean,R_5_std,R_5_min,R_5_max,D_58_last,D_58_mean,D_58_std,D_58_min,D_58_max,S_9_last,S_9_mean,S_9_std,S_9_min,S_9_max,B_14_last,B_14_mean,B_14_std,B_14_min,B_14_max,D_59_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_60_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_61_last,D_61_mean,D_61_std,D_61_min,D_61_max,B_15_last,B_15_mean,B_15_std,B_15_min,B_15_max,S_11_last,S_11_mean,S_11_std,S_11_min,...,D_88_last_round2,B_31_last_round2,S_19_last_round2,R_19_last_round2,B_32_last_round2,S_20_last_round2,R_20_last_round2,R_21_last_round2,B_33_last_round2,D_89_last_round2,R_22_last_round2,R_23_last_round2,D_91_last_round2,D_92_last_round2,D_93_last_round2,D_94_last_round2,R_24_last_round2,R_25_last_round2,D_96_last_round2,S_22_last_round2,S_23_last_round2,S_24_last_round2,S_25_last_round2,S_26_last_round2,D_102_last_round2,D_103_last_round2,D_104_last_round2,D_105_last_round2,D_106_last_round2,D_107_last_round2,B_36_last_round2,B_37_last_round2,R_26_last_round2,R_27_last_round2,D_108_last_round2,D_109_last_round2,D_110_last_round2,D_111_last_round2,B_39_last_round2,D_112_last_round2,B_40_last_round2,S_27_last_round2,D_113_last_round2,D_115_last_round2,D_118_last_round2,D_119_last_round2,D_121_last_round2,D_122_last_round2,D_123_last_round2,D_124_last_round2,D_125_last_round2,D_127_last_round2,D_128_last_round2,D_129_last_round2,B_41_last_round2,B_42_last_round2,D_130_last_round2,D_131_last_round2,D_132_last_round2,D_133_last_round2,R_28_last_round2,D_134_last_round2,D_135_last_round2,D_136_last_round2,D_137_last_round2,D_138_last_round2,D_139_last_round2,D_140_last_round2,D_141_last_round2,D_142_last_round2,D_143_last_round2,D_144_last_round2,D_145_last_round2,P_2_last_mean_diff,D_39_last_mean_diff,B_1_last_mean_diff,B_2_last_mean_diff,R_1_last_mean_diff,S_3_last_mean_diff,D_41_last_mean_diff,B_3_last_mean_diff,D_42_last_mean_diff,D_43_last_mean_diff,D_44_last_mean_diff,B_4_last_mean_diff,D_45_last_mean_diff,B_5_last_mean_diff,R_2_last_mean_diff,D_46_last_mean_diff,D_47_last_mean_diff,D_48_last_mean_diff,D_49_last_mean_diff,B_6_last_mean_diff,B_7_last_mean_diff,B_8_last_mean_diff,D_50_last_mean_diff,D_51_last_mean_diff,B_9_

In [13]:
# train.to_csv("../tmp/train_agg_v2.csv", header=True, index=False)
# test.to_csv("../tmp/test_agg_v2.csv", header=True, index=False)

In [14]:
params = {
    'objective': 'binary',
    'metric': CFG.metric,
    'boosting': CFG.boosting_type,
    'seed': CFG.seed,
    'num_leaves': 100,
    'learning_rate': 0.05,
    'feature_fraction': 0.20,
    'bagging_freq': 10,
    'bagging_fraction': 0.50,
    'n_jobs': -1,
    'lambda_l2': 2,
    'min_data_in_leaf': 40,
    "num_iterations": 3000,
    }

test_predictions = np.zeros(len(test))
oof_predictions = np.zeros(len(train))

kfold = StratifiedKFold(n_splits = CFG.n_folds, 
                        shuffle = True, 
                        random_state = CFG.seed)

for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold} with {len(features)} features...')
    
    x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
    y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
        
    model = lgb.LGBMClassifier(**params)
    model.fit(x_train, y_train, 
              categorical_feature=cat_features, 
              eval_set=[(x_val, y_val)], 
              eval_metric="binary_logloss",
              early_stopping_rounds=1000, 
              verbose=1000)
    

    val_pred = model.predict(x_val)

    oof_predictions[val_ind] = val_pred
    
    test_pred = model.predict(test[features])
    test_predictions += test_pred / CFG.n_folds
    
    score = amex_metric(y_val, val_pred)
    print(f'Our fold {fold} CV score is {score}')
    del x_train, x_val, y_train, y_val
    gc.collect()

 
--------------------------------------------------
Training fold 0 with 1427 features...
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[1000]	valid_0's binary_logloss: 0.215662
[2000]	valid_0's binary_logloss: 0.214874
[3000]	valid_0's binary_logloss: 0.215819
Our fold 0 CV score is 0.5892265747691119
 
--------------------------------------------------
Trainin

In [17]:
score = amex_metric(train[CFG.target], oof_predictions)
print(f'Our out of folds CV score is {score}')

Our out of folds CV score is 0.5856610815488341


In [16]:
for feature_name, feature_imp in sorted(zip(model.feature_name(), model.feature_importance()), 
                                        key=lambda x: -x[1]):
    print(feature_imp, feature_name)

AttributeError: 'LGBMClassifier' object has no attribute 'feature_name'

In [18]:

import sys
sys.path.append("..")
from src import *

test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 
                        'prediction': test_predictions})

        
save_submission(test_df, 
                name=f"ldbm_dart_final_cv_score_{score}", 
                submit_on_kaggle=True)

telegram(f"Submit done")

b'Successfully submitted to American Express - Default Prediction'
